In [1]:
!pip install transformers datasets tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 9.8 MB/s 
     |████████████████████████████████| 362 kB 51.6 MB/s 
     |████████████████████████████████| 6.6 MB 52.1 MB/s 
     |████████████████████████████████| 596 kB 70.6 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 212 kB 51.7 MB/s 
     |████████████████████████████████| 140 kB 93.0 MB/s 
     |████████████████████████████████| 1.1 MB 68.0 MB/s 
     |████████████████████████████████| 127 kB 77.6 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 271 kB 95.5 MB/s 
     |████████████████████████████████| 144 kB 69.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing

A Q&A problem is generally defined as an NLP problem with a given text and a question of AI and getting an answer back. usually, this answer can be found in the original text but there are various approaches to this problem. In the case of visual question answering (VQA), the question is about a visual entity or concept rather than text but the question itself is in the form of text.

Most of the models that are intended to be used inVQA are multimodal models that can understand the visual content along with the question and geneate the answer properly. But, unimodal fully textual QA or just QA is based on textual context and textual questions with respective textual answers.

SQUAD is one of the most well-known datasets in the field of QA. Next we will implement some of the code example of SQUAD and examine them.

In [2]:
from pprint import pprint 
from datasets import load_dataset 
squad = load_dataset("squad") 
for item in squad["train"][1].items(): 
    print(item[0]) 
    pprint(item[1]) 
    print("="*20)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

id
'5733be284776f4190066117f'
title
'University_of_Notre_Dame'
context
('Architecturally, the school has a Catholic character. Atop the Main '
 "Building's gold dome is a golden statue of the Virgin Mary. Immediately in "
 'front of the Main Building and facing it, is a copper statue of Christ with '
 'arms upraised with the legend "Venite Ad Me Omnes". Next to the Main '
 'Building is the Basilica of the Sacred Heart. Immediately behind the '
 'basilica is the Grotto, a Marian place of prayer and reflection. It is a '
 'replica of the grotto at Lourdes, France where the Virgin Mary reputedly '
 'appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive '
 '(and in a direct line that connects through 3 statues and the Gold Dome), is '
 'a simple, modern stone statue of Mary.')
question
'What is in front of the Notre Dame Main Building?'
answers
{'answer_start': [188], 'text': ['a copper statue of Christ']}


However, there is a version 2 of the squad dataset, which has more training smaples, and it is highly recommended to use it. To have an overall understanding of how it is possible to train a model for a QA problem, we will focus on the current part of this problem.

Next we will write some codes to implement the version 2.

In [3]:
squad = load_dataset("squad_v2")
squad

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

As can be seen from the above result there are more than 130000 training samples with more than 11000 validation samples.


As we did before, we must preprocess the data to have the right form to be used by the model. To do so, we must first load our tokenizer which is pretrained tokenizer as long as we are using a pretrained model adn we want to fine-tune it for a QA problem.

In [4]:
from transformers import AutoTokenizer
model = "distilbert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(model)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

As can be seen from the above result, we area going to use the distillBERT model. According to SQUAD example, we will need to give more than 1 text to the model, one for the questiona dn one for the context. Accordingly, we need our tokenizer to put these 2 side by side and seperate them with the special [SEP] tiken because distillBERT is a BERT based model.

There is another problem in teh scope of QA and it is the size of the context. The content size can be longer than the model input size but we can not reduce it to the size the model accepts. With some problems we can do so but in AQ it is possible that the answer could be in the truncated part.

In [5]:
max_length = 384 
doc_stride = 128 
example = squad["train"][173] 
tokenized_example = tokenizer( 
example["question"], 
example["context"], 
max_length=max_length, 
truncation="only_second", 
return_overflowing_tokens=True, 
stride=doc_stride 
)

The stride is the document stride used to return the stride for the second part like a window, while the return_overflowing_tokens flag gives the model information on whether it should return the extra tokens. The result of tokenized_example is more than a single tkenized output, instead having 2 input IDs.

In [6]:
# len of the tokenized example
len(tokenized_example['input_ids'])

2

In [7]:
# We can see the whole result by using for loop
for input_ids in tokenized_example["input_ids"][:2]: 
    print(tokenizer.decode(input_ids)) 
    print("-"*50)

[CLS] beyonce got married in 2008 to whom? [SEP] on april 4, 2008, beyonce married jay z. she publicly revealed their marriage in a video montage at the listening party for her third studio album, i am... sasha fierce, in manhattan's sony club on october 22, 2008. i am... sasha fierce was released on november 18, 2008 in the united states. the album formally introduces beyonce's alter ego sasha fierce, conceived during the making of her 2003 single " crazy in love ", selling 482, 000 copies in its first week, debuting atop the billboard 200, and giving beyonce her third consecutive number - one album in the us. the album featured the number - one song " single ladies ( put a ring on it ) " and the top - five songs " if i were a boy " and " halo ". achieving the accomplishment of becoming her longest - running hot 100 single in her career, " halo "'s success in the us helped beyonce attain more top - ten singles on the list than any other woman during the 2000s. it also included the suc

As can be seen from the above output , with a window of 128 tokens, the rest of the context is replicated again in the second output of input IDs

Another problem is the end span, which is not available in the dataset but instead, the start span or the start character for the answer is given. it is easy to find the legnth of the answer and add it the start span, which will automatically yield the end span.

In [8]:
def prepare_train_features(examples, pad_on_right=True): 
    tokenized_examples = tokenizer( 
        examples["question" if pad_on_right else "context"], 
        examples["context" if pad_on_right else "question"], 
        truncation="only_second" if pad_on_right else "only_first", 
        max_length=max_length, 
        stride=doc_stride, 
        return_overflowing_tokens=True, 
        return_offsets_mapping=True, 
        padding="max_length", 
    ) 
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping") 
    offset_mapping = tokenized_examples.pop("offset_mapping") 
    tokenized_examples["start_positions"] = [] 
    tokenized_examples["end_positions"] = [] 
    for i, offsets in enumerate(offset_mapping): 
        input_ids = tokenized_examples["input_ids"][i] 
        cls_index = input_ids.index(tokenizer.cls_token_id) 
        sequence_ids = tokenized_examples.sequence_ids(i) 
        sample_index = sample_mapping[i] 
        answers = examples["answers"][sample_index] 
        if len(answers["answer_start"]) == 0: 
            tokenized_examples["start_positions"].append(cls_index) 
            tokenized_examples["end_positions"].append(cls_index) 
        else: 
            start_char = answers["answer_start"][0] 
            end_char = start_char + len(answers["text"][0]) 
            token_start_index = 0 
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0): 
                token_start_index += 1 
            token_end_index = len(input_ids) - 1 
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0): 
                token_end_index -= 1 
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char): 
                tokenized_examples["start_positions"].append(cls_index) 
                tokenized_examples["end_positions"].append(cls_index) 
            else: 
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char: 
                    token_start_index += 1 
                tokenized_examples["start_positions"].append(token_start_index - 1) 
                while offsets[token_end_index][1] >= end_char: 
                    token_end_index -= 1 
                tokenized_examples["end_positions"].append(token_end_index + 1) 
    return tokenized_examples

In [9]:
# Map this function to the dataset would apply all the required changes
tokenized_datasets = squad.map(prepare_train_features, batched=True, remove_columns=squad["train"].column_names)

Parameter 'function'=<function prepare_train_features at 0x7f0f85f91d40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [10]:
# Now it is a time to load a pretrained model to be fine-tuned
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer 
model = AutoModelForQuestionAnswering.from_pretrained(model)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [11]:
# Creating training arugments
args = TrainingArguments( 
f"test-squad", 
evaluation_strategy = "epoch", 
learning_rate=2e-5, 
per_device_train_batch_size=16, 
per_device_eval_batch_size=16, 
num_train_epochs=3, 
weight_decay=0.01, 
)

If we are not going to use a data collator, we will give a defaut data collator to the model trainer.

In [12]:
from transformers import default_data_collator
data_collator= default_data_collator

In [13]:
# Create a trainer.
trainer = Trainer( 
model, 
args, 
train_dataset=tokenized_datasets["train"], 
eval_dataset=tokenized_datasets["validation"], 
data_collator=data_collator, 
tokenizer=tokenizer, 
)

In [14]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 131754
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 24705


Epoch,Training Loss,Validation Loss
1,1.236900,1.259186
2,0.972300,1.256190
3,0.779100,1.430450


Saving model checkpoint to test-squad/checkpoint-500
Configuration saved in test-squad/checkpoint-500/config.json
Model weights saved in test-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-squad/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-squad/checkpoint-1000
Configuration saved in test-squad/checkpoint-1000/config.json
Model weights saved in test-squad/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-squad/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-squad/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-squad/checkpoint-1500
Configuration saved in test-squad/checkpoint-1500/config.json
Model weights saved in test-squad/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-squad/checkpoint-1500/tokenizer_config.json
Special tokens file saved in test-squad/checkpoint-15

TrainOutput(global_step=24705, training_loss=1.1039067681997177, metrics={'train_runtime': 8061.0335, 'train_samples_per_second': 49.034, 'train_steps_per_second': 3.065, 'total_flos': 3.873165421863629e+16, 'train_loss': 1.1039067681997177, 'epoch': 3.0})

As can be seen from the above result, the model is trained with 3 epochs and the ouputs for loss in validation and training are reported

In [15]:
# Save the model
trainer.save_model("distillBERT_SQUAD")

Saving model checkpoint to distillBERT_SQUAD
Configuration saved in distillBERT_SQUAD/config.json
Model weights saved in distillBERT_SQUAD/pytorch_model.bin
tokenizer config file saved in distillBERT_SQUAD/tokenizer_config.json
Special tokens file saved in distillBERT_SQUAD/special_tokens_map.json


If we want to use our saved model or any other model that is trained on QA, the transformers library provides a pipline that is easy to use amd implment with no extra effort.

By using the below pipeline functionary we can use any model. Now it is a time to implement this.

In [16]:
from transformers import pipeline
qa_model = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='distilbert-base-cased')

https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmprdvkk6s0


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3
creating metadata file for /root/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3
loading configuration file https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased-distilled-squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/02f483764e26e1acfddfcd6e4879785f2908b2806a962d01b888bfe2b988075b.bd96a9432b167ab5e2b086cf0b688ca3c43c027091377eddfcfd39bdde851c35
creating metadata file for /root/.cache/huggingface/transformers/02f483764e26e1acfddfcd6e4879785f2908b2806a962d01b888bfe2b988075b.bd96a9432b167ab5e2b086cf0b688ca3c43c027091377eddfcfd39bdde851c35
loading weights file https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/02f483764e26e1acfddfcd6e4879785f2908b2806a962d01b888bfe2b988075b.bd96a9432b167ab5e2b086cf0b688ca3c43c027091377eddfcfd39bdde851c35
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were initialized from the model checkpoint at distilbert-base-cased-distilled-squad.
If yo

...

storing https://huggingface.co/distilbert-base-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a
creating metadata file for /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a
loading configuration file https://huggingface.co/distilbert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distil

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/ba377304984dc63e3ede0e23a938bbbf04d5c3835b66d5bb48343aecca188429.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
creating metadata file for /root/.cache/huggingface/transformers/ba377304984dc63e3ede0e23a938bbbf04d5c3835b66d5bb48343aecca188429.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
https://huggingface.co/distilbert-base-cased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpor87pqam


Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/acb5c2138c1f8c84f074b86dafce3631667fccd6efcb1a7ea1320cf75c386a36.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
creating metadata file for /root/.cache/huggingface/transformers/acb5c2138c1f8c84f074b86dafce3631667fccd6efcb1a7ea1320cf75c386a36.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
loading file https://huggingface.co/distilbert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/ba377304984dc63e3ede0e23a938bbbf04d5c3835b66d5bb48343aecca188429.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/distilbert-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/acb5c2138c1f8c84f074b86dafce3631667fccd6efcb1a7ea1320cf75c386a36.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
loading file 

The pipeline just needs 2 inputs to make the model ready for usage, the model adn the tokenizer. Although we are alsorequired to give it a pipeline type which is QA in this project. The next step is to give it the input it requires. And these inputs will be context and question.

In [17]:
question=squad["validation"][0]["question"]
context = squad["validation"][0]["context"]
print("Question: ", question)
print("Context: ", context)

Question:  In what country is Normandy located?
Context:  The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.


In [18]:
qa_model(question=question, context=context)

{'answer': 'France', 'end': 165, 'score': 0.9889376759529114, 'start': 159}

We did learned how we can train on the dataset that we want and we also learned how to use the pipelines to train the model.